# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread
from skimage.transform import resize
import datetime
import os



We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 60 #experiment with the batch size
# batch size should be the multiplcation of no of frame

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
img_row,img_col = 120,120
img_idx = [n for n in range(0, 30, 2)] #create a list of image numbers you want to use for a particular video
x = len(img_idx) 
y, z = img_row, img_col

In [5]:
print(img_idx)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]


In [6]:
def generator(source_path, folder_list, batch_size):
    while True:
        t = np.random.permutation(folder_list)
        total_no_frame=len(folder_list)*x
        num_batches = total_no_frame//batch_size #number of batches
        for batch in range(num_batches): # iterate over the number of batches
            batch_itr=0
            batch_data = np.zeros((batch_size,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size//x): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*(batch_size//x))].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*(batch_size//x))].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
    
                    
#                     image = image.resize(250,250) ##resize
                    # crop the images and resize them. Note that the images are of 2 different shape 
                    # and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[0] > 100 and image.shape[1] > 100:
#                         y,z = 120,120
                        no_rows,no_cols,no_chs=image.shape
                        if no_rows == 120 and no_cols == 160:
                            image = image[0:120,20:140]
                        elif no_rows == 360 and no_cols == 360:
                            image = image[0:360:3,0:360:3]
#                     print("image:",len(image))
#                     batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                    batch_data[batch_itr,:,:,0] = image[:, : , 0]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[batch_itr,:,:,1] = image[:, : , 1]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[batch_itr,:,:,2] = image[:, : , 2]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_labels[batch_itr, int(t[folder + (batch*(batch_size//x))].strip().split(';')[2])] = 1
                    batch_itr=batch_itr+1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (total_no_frame%batch_size != 0):
            batch_size = (total_no_frame- num_batches*batch_size)
            batch_data = np.zeros((batch_size,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            batch_itr=0
            for folder in range(batch_size//x): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*(batch_size//x))].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder +  (batch*(batch_size//x))].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    no_rows,no_cols,no_chs=image.shape
                    if image.shape[0] > 100 and image.shape[1] > 100:
#                         y,z = 120,120            
                        if no_rows == 120 and no_cols == 160:
                            image = image[0:120,20:140]
                        elif no_rows == 360 and no_cols == 360:
                            image = image[0:360:3,0:360:3]
#                     print("image:",len(image))
#                     batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                    batch_data[batch_itr,:,:,0] = image[:, : , 0]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[batch_itr,:,:,1] = image[:, : , 0]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_data[batch_itr,:,:,2] = image[:, : , 0]/255#(image - np.min(image))/(np.max(image)-np.min(image))#normalise and feed in the image
                    batch_labels[batch_itr, int(t[folder + (batch*(batch_size//x))].strip().split(';')[2])] = 1
                    batch_itr=batch_itr+1
            
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)*x
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)*x
print('# validation sequences =', num_val_sequences)
num_epochs = 2# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 9945
# validation sequences = 1500
# epochs = 2


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [40]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
# using resnet 18
# !pip install resnet
# import resnet
# from keras.applications import ResNet50
# resnet = ResNet50(weights='imagenet', include_top=False)
# model = resnet.ResnetBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes)
# model.compile(loss='categorical_crossentropy', optimizer='SGD',
#               metrics=['accuracy'])

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
# x,y,z = 30,250,250
# x,y,z = 10,100,100
input_shape=(y,z,3)

# define CNN model

model=Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same'), input_shape=(len(img_idx), 120, 120, 3)))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(32, (3, 3))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dropout(0.25)))


model.add(TimeDistributed(Dense(512)))

model.add(TimeDistributed(Dense(5)))

model.add(GRU(256,dropout=0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [33]:
optimiser = optimizers.Adam()#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_94 (TimeDis (None, 15, 120, 120, 32)  896       
_________________________________________________________________
time_distributed_95 (TimeDis (None, 15, 120, 120, 32)  0         
_________________________________________________________________
time_distributed_96 (TimeDis (None, 15, 118, 118, 32)  9248      
_________________________________________________________________
time_distributed_97 (TimeDis (None, 15, 118, 118, 32)  0         
_________________________________________________________________
time_distributed_98 (TimeDis (None, 15, 59, 59, 32)    0         
_________________________________________________________________
time_distributed_99 (TimeDis (None, 15, 111392)        0         
_________________________________________________________________
time_distributed_100 (TimeDi (None, 15, 111392)        0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [35]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [36]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [37]:
print(steps_per_epoch,validation_steps, num_epochs )

166 25 2


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [38]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/2


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]


ValueError: Error when checking input: expected time_distributed_94_input to have 5 dimensions, but got array with shape (60, 120, 120, 3)

In [ ]:
#model.summary()